## Movie Recommendation
### 프로젝트 - Movielens 영화 추천 실습
***
> **1) 데이터 준비와 전처리
> Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.**  
> 
> **2) 분석해 봅시다.  
> -ratings에 있는 유니크한 영화 개수  
> -rating에 있는 유니크한 사용자 수  
> -가장 인기 있는 영화 30개(인기순)**  
> 
> **3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.**  
> **4) CSR matrix를 직접 만들어 봅시다.**  
> **5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.**  
> **6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.**  
> **7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.**  
> **8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.**
***
**평가문항**  
> ***1. CSR matrix가 정상적으로 만들어졌다.  
> -사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.***  
>  
> ***2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.   
> -사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.***  
>  
> ***3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
> -MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.***  
***
**1) 데이터 준비와 전처리**

In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings에서 timestamp 컬럼 삭제
ratings.drop(['timestamp'], axis=1, inplace=True)

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**2) 분석해 봅시다.**
**-ratings에 있는 유니크한 영화 개수
-rating에 있는 유니크한 사용자 수
-가장 인기 있는 영화 30개(인기순)**

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자수
ratings['user_id'].nunique()

6039

In [8]:
# 합쳐주기
ratings_title= pd.merge(ratings, movies[['title','movie_id']], on='movie_id', how='left')
ratings_title

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)


In [9]:
# 데이터에 영화이름을 넣어도 되지않을까?
# 영화이름 넣었을때, 학습시 문제는?
# 포켓몬스터 참고하기

In [10]:
# ratings에 가장 인기많은 영화
movie_count = ratings_title.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 내가 좋아하는 영화 5개 고르기
my_favorite = ['Home Alone (1990)' , 'Snow White and the Seven Dwarfs (1937)' ,'Aladdin (1992)' ,'Pinocchio (1940)' ,'Beauty and the Beast (1991)']

# 'ethan'이라는 user_id가 위영화의 평가점수를 4점씩 주었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['ethan']*5, 'title': my_favorite, 'count':[4]*5})

if not ratings_title.isin({'user_id':['ethan']})['user_id'].any():  # user_id에 'ethan'이라는 데이터가 없다면
    ratings_title = ratings_title.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings_title.tail(10) 

,user_id,movie_id,count,title
836473,6040,1090.0,3,Platoon (1986)
836474,6040,1094.0,5,"Crying Game, The (1992)"
836475,6040,562.0,5,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,Sophie's Choice (1982)
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982)
0,ethan,NaN,4,Home Alone (1990)
1,ethan,NaN,4,Snow White and the Seven Dwarfs (1937)
2,ethan,NaN,4,Aladdin (1992)
3,ethan,NaN,4,Pinocchio (1940)
4,ethan,NaN,4,Beauty and the Beast (1991)


In [12]:
# index번호를 다시 매겨준다
ratings_title.reset_index(drop=True, inplace=True)
ratings_title.tail(10)

,user_id,movie_id,count,title
836473,6040,1090.0,3,Platoon (1986)
836474,6040,1094.0,5,"Crying Game, The (1992)"
836475,6040,562.0,5,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,Sophie's Choice (1982)
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982)
836478,ethan,NaN,4,Home Alone (1990)
836479,ethan,NaN,4,Snow White and the Seven Dwarfs (1937)
836480,ethan,NaN,4,Aladdin (1992)
836481,ethan,NaN,4,Pinocchio (1940)
836482,ethan,NaN,4,Beauty and the Beast (1991)


In [13]:
ratings_title_num= pd.merge(ratings_title, movies[['title','movie_id']], on='title', how='left')
ratings_title_num.tail(10)

,user_id,movie_id_x,count,title,movie_id_y
836473,6040,1090.0,3,Platoon (1986),1090
836474,6040,1094.0,5,"Crying Game, The (1992)",1094
836475,6040,562.0,5,Welcome to the Dollhouse (1995),562
836476,6040,1096.0,4,Sophie's Choice (1982),1096
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,NaN,4,Home Alone (1990),586
836479,ethan,NaN,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,NaN,4,Aladdin (1992),588
836481,ethan,NaN,4,Pinocchio (1940),596
836482,ethan,NaN,4,Beauty and the Beast (1991),595


In [14]:
# ratings_title_num에서 movie_id_x 컬럼 삭제
ratings_title_num.drop(['movie_id_x'], axis=1, inplace=True)
# rating 컬럼의 이름을 count로 바꿉니다.
ratings_title_num.rename(columns={'movie_id_y':'movie_id'}, inplace=True)

In [15]:
ratings_title_num.tail(10)

,user_id,count,title,movie_id
836473,6040,3,Platoon (1986),1090
836474,6040,5,"Crying Game, The (1992)",1094
836475,6040,5,Welcome to the Dollhouse (1995),562
836476,6040,4,Sophie's Choice (1982),1096
836477,6040,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,4,Home Alone (1990),586
836479,ethan,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,4,Aladdin (1992),588
836481,ethan,4,Pinocchio (1940),596
836482,ethan,4,Beauty and the Beast (1991),595


In [16]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings_title_num['user_id'].unique()
movie_unique = ratings_title_num['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
print(user_to_idx['ethan'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['Aladdin (1992)'])

6039
33


In [18]:
ratings_title_num.tail(10)

,user_id,count,title,movie_id
836473,6040,3,Platoon (1986),1090
836474,6040,5,"Crying Game, The (1992)",1094
836475,6040,5,Welcome to the Dollhouse (1995),562
836476,6040,4,Sophie's Choice (1982),1096
836477,6040,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,4,Home Alone (1990),586
836479,ethan,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,4,Aladdin (1992),588
836481,ethan,4,Pinocchio (1940),596
836482,ethan,4,Beauty and the Beast (1991),595


In [19]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings_title_num['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_title_num):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings_title_num['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings_title_num['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings_title_num):
    print('amovie column indexing OK!!')
    ratings_title_num['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings_title_num

user_id column indexing OK!!
amovie column indexing OK!!


,user_id,count,title,movie_id
0,0,5,0,1193
1,0,3,1,661
2,0,3,2,914
3,0,4,3,3408
4,0,5,4,2355
...,...,...,...,...
836478,6039,4,507,586
836479,6039,4,8,594
836480,6039,4,33,588
836481,6039,4,528,596


In [26]:
ratings_title_num['count']

0         5
1         3
2         3
3         4
4         5
         ..
836478    4
836479    4
836480    4
836481    4
836482    4
Name: count, Length: 836483, dtype: int64

In [38]:
print(ratings_title_num['count'].values)

[5 3 3 ... 4 4 4]


In [39]:
from scipy.sparse import csr_matrix

num_user = ratings_title_num['user_id'].nunique()
num_movie = ratings_title_num['title'].nunique()

csr_data = csr_matrix((ratings_title_num['count'].values, (ratings_title_num.user_id, ratings_title_num.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [40]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [41]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [42]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [43]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [44]:
ethan, aladdin = user_to_idx['ethan'], movie_to_idx['Aladdin (1992)']
ethan_vector, aladdin_vector = als_model.user_factors[ethan], als_model.item_factors[aladdin]

In [45]:
ethan_vector

array([-0.16464461,  0.17881715, -0.18447036,  0.75373286, -0.17607324,
        0.10237523,  0.2656933 ,  0.7354863 ,  0.20405762, -0.27282065,
       -0.51300555, -0.7647271 ,  0.00619444,  0.5370067 ,  0.15553701,
       -0.282316  , -0.34134424, -0.5117699 , -0.2868811 ,  0.3734475 ,
        0.4550405 , -0.16628139,  0.5714116 ,  0.34759086, -0.29997435,
       -0.01305543, -0.17150015,  0.13396406,  0.17544918, -0.1395754 ,
       -0.56159264,  0.12927157,  0.32613474, -0.14162782, -0.2725848 ,
        0.12680033,  0.06322315, -0.4805928 ,  0.2900136 , -0.11153615,
       -0.30626383,  0.16476445, -0.1586299 ,  0.6791488 ,  0.29452655,
       -0.1587098 ,  0.12871121, -0.3092393 , -0.35272086,  0.08896325,
       -0.19890557,  0.5468729 , -0.36146933,  1.1955005 ,  0.60269684,
       -0.2544454 ,  0.43211374,  0.9582275 , -0.611201  ,  0.15071012,
       -0.03942791,  0.06994128,  0.26131546, -0.18451503,  0.42861718,
        0.67777866, -0.0899891 , -0.13600244,  0.14261143, -0.50

In [47]:
aladdin_vector

array([ 0.00297128,  0.0084359 , -0.00615836,  0.03744302, -0.01414737,
       -0.00034189, -0.01799412,  0.03409342,  0.01149173, -0.0226124 ,
        0.01550529, -0.00585469,  0.00493539,  0.01831236,  0.02115725,
       -0.00302313, -0.00425732, -0.01167858, -0.01772211,  0.02037171,
        0.01046041,  0.00173459,  0.02487434,  0.02231075, -0.00797964,
        0.00386762, -0.01254574,  0.01183014,  0.01522909,  0.00125052,
        0.00333736,  0.01232229,  0.03360846, -0.0021018 ,  0.01672683,
        0.01708404,  0.00808978, -0.01794077,  0.0149743 , -0.01007996,
       -0.00496741,  0.01286469,  0.01976074,  0.04382332,  0.00277839,
        0.03924133,  0.00118226, -0.00697851,  0.01777453, -0.0067477 ,
       -0.01340156,  0.01180358, -0.0258846 ,  0.05434312, -0.00143128,
       -0.00919949,  0.00750235,  0.04731474, -0.0246823 ,  0.02695708,
        0.0025502 ,  0.0235714 ,  0.00673633, -0.00946562,  0.00792059,
        0.04313264,  0.01359271, -0.0016138 ,  0.00383508, -0.00

In [48]:
np.dot(ethan_vector, aladdin_vector)

0.54678357